### Closures

let's examine that concept of a cell to create an indirect reference for variables that are in multiple scopes

In [1]:
def outer():
    x = 'python'

    def inner():
        print(x)
    
    return inner

In [2]:
fn = outer()

In [3]:
fn

<function __main__.outer.<locals>.inner()>

In [4]:
fn.__code__.co_freevars

('x',)

In [5]:
fn.__closure__

(<cell at 0x106196230: str object at 0x1040abef0>,)

As we can see, x is a free variable in the closure

In [6]:
fn.__closure__

(<cell at 0x106196230: str object at 0x1040abef0>,)

Here we see that the free variable x is actually a reference to a cell object that is itself a reference to a string object.

Let's see what the memory address of x is in the outer func and the inner func.

To be sure string interning dones not play a role,  

In [7]:
def outer():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    
    def inner():
        print('inner:', hex(id(x)))
        print(x)
    return inner

In [8]:
fn = outer()

outer: 0x106604840


In [12]:
fn.__code__.co_freevars


('x',)

In [11]:
fn.__closure__

(<cell at 0x106197c40: list object at 0x106604840>,)

In [13]:
fn()

inner: 0x106604840
[1, 2, 3]


As you can see, each the memory address of x in outer, inner and the cell all point to the smae object.

#### Modifying the Free Variable

We know we can  odify nonlocal variables by using the nonlocal keywords. So the following will work:

In [14]:
def counter():
    count = 0 # local variable
    
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [15]:
c = counter()

In [16]:
c()

1

In [17]:
c()

2

In [19]:
c()

3

In [20]:
counter()

<function __main__.counter.<locals>.inc()>

#### Sahred Extended Scopes

As we saw in the lecture, we can setup nonlocal variables in different inner funct ions that reference the same outer scope variable, ie. we have a free variable that is shared between two closures. This works because both non local variables and the outer local variable all point6 back to the same cell object.

In [21]:
def outer():
    count = 0
    
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count

    return inc1, inc2

In [22]:
fn1, fn2 = outer()

In [23]:
fn1.__closure__, fn2.__closure__

((<cell at 0x105f3b6a0: int object at 0x10299af90>,),
 (<cell at 0x105f3b6a0: int object at 0x10299af90>,))

In [24]:
fn1()

1

In [25]:
fn1()

2

In [26]:
fn2()

3

Multiple Instances of Closures

Recall that every time a function is called, a new local scope is created~~~~

In [27]:
from time import perf_counter

def func():
    x = perf_counter()
    print(x, id(x))

In [28]:
func()

11307.2821065 4587172112


In [29]:
func()

11312.4340215 4587171984


#### Same thing happens with Closures

They have their own extended scope every time the closure is created:

In [43]:
def pow(n):
    
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    
    return inner

In [44]:
square = pow(2)

In [45]:
square, square.__closure__, square.__code__.co_freevars

(<function __main__.pow.<locals>.inner(x)>,
 (<cell at 0x1067343a0: int object at 0x10299afd0>,),
 ('n',))

In [46]:
square(5)

25

In [47]:
cube = pow(3)

In [48]:
cube(5)

125

In [49]:
square.__closure__

(<cell at 0x1067343a0: int object at 0x10299afd0>,)

In [50]:
cube.__closure__

(<cell at 0x1067423e0: int object at 0x10299aff0>,)

In [51]:
square.__code__.co_freevars, cube.__code__.co_freevars

(('n',), ('n',))

#### 注意！

Rember when we said the captured variable is a reference established when the closure is created, but the value is looked up only once the function is called..

This could create very subtle bugs in our program!!!

Consider the following example where we want to create some fun ctions that can add 1, 2, 3, 4 and to whatever is passed to them.

In [52]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [53]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [54]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

#### 但如果我們想要寫得炫一些：

In [55]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [56]:
adders = create_adders()

In [57]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

In [58]:
adders[3](10)

14

In [61]:
adders[2](10)

14

In [60]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(14, 14, 14, 14)

#### Nested Closures

We can also next closures, as can be seen in this example:


In [63]:
def incrementer(n):
    def inner(start):
        current = start
        def inc():
            a = 10 # local var
            nonlocal current
            current += n
            return current
        return inc
    return inner

In [ ]:
fn = incrementer(2)

In [65]:
fn

<function __main__.incrementer.<locals>.inner(start)>

In [66]:
fn.__closure__, fn.__code__.co_freevars

((<cell at 0x106752770: int object at 0x10299afd0>,), ('n',))

In [67]:
inc_2 = fn(100)

In [69]:
inc_2

<function __main__.incrementer.<locals>.inner.<locals>.inc()>

In [70]:
inc_2.__code__.co_freevars

('current', 'n')

> Aaron's Experiments with `Closures`

<font color=#d08770>#d08770


<font color=#d08770>Let's examine that concept of a cell to create an indirect reference for variables that are in multiple scopes.

In [2]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [3]:
fn = outer()

In [4]:
fn.__code__.co_freevars

('x',)

<font color=#d08770>As we can see, `x` is a `free variable` in the `closure`

In [5]:
fn.__closure__

(<cell at 0x1133aaf80: str object at 0x110624230>,)

<font color=#d08770>Here we see that the **free variable x is actually a reference to a cell object** that is itself a reference to a string object.

Let's see what the memory address of `x` is in the outer function and the inner function. 

To be sure string interning does not play a role, we are going to use an object that we know Python will not automatically intern, like a list.

In [9]:
x = 'hello'
hex(id(x))

'0x112cce230'

In [10]:
print(hex(id(x)))

0x112cce230


In [15]:
def outer():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    def inner():
        print('inner:', hex(id(x)))
        print(x)
    return inner

In [17]:
fn = outer()

outer: 0x113502c40


In [13]:
fn.__closure__

(<cell at 0x11338fca0: list object at 0x1133dfb80>,)

In [18]:
fn()

inner: 0x113502c40
[1, 2, 3]


<font color=#d08770>注意，如果只有執行function的定義，而沒有運行，closure裡面的free variables是不會有內容的：

In [21]:
def outer2():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    def inner2():
        print('inner:', hex(id(x)))
        print(x)
    return inner2

In [22]:
fn2 = outer2

In [24]:
fn2()

outer: 0x11353acc0


<function __main__.outer2.<locals>.inner2()>

In [25]:
fn2()

outer: 0x11353b000


<function __main__.outer2.<locals>.inner2()>

以上的fn2不管執行多少次，x裡面都沒有內容

<font color=#d08770> As you can see, each memory address of `x` in `outer`, `inner` and the cell all point to the same object.

#### <font color=#a3be8c> Modifying the Free Variable 

We know we can modify nonlocal variables by using the `nonlocal` keyword. So the following will work:

In [26]:
def counter():
    count = 0 # local variable
    
    def inc():
        nonlocal count # this is the count variable
        count += 1
        return count
    return inc

In [27]:
c = counter()

In [28]:
c()

1

In [29]:
c()

2

<font color=#d08770> Shared Extended Scopes
ter local variable all point b  the same cell object.ack to
We can setup nonlocal variables in different inner functions that reference the same outer scope varible, i.e. we have a free variable that is shared between two closures. This works because both nonlocal variables and the outer local variable all point back to nonlocal variables and the outer local variable all point back to the same cell object.

In [30]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

In [31]:
fn1, fn2 = outer()

In [32]:
fn1.__closure__, fn2.__closure__

((<cell at 0x11338fbb0: int object at 0x104ad6f90>,),
 (<cell at 0x11338fbb0: int object at 0x104ad6f90>,))

<font color=#bf616a>As you can see here, the `count` label points to the same cell.

In [33]:
fn1()

1

In [34]:
fn1()

2

In [35]:
fn2()

3

### <font color=#a3be8c> Multiple Instances of Closures

Recall that every time a function is called, a new local scope is created

In [36]:
from time import perf_counter

def func():
    x = perf_counter()
    print(x, id(x))

In [37]:
func()

83184.872749375 4616047280


In [38]:
func()

83234.104057416 4616046768


<font color=#88c0d0>The same thing happens with closures, they have their own extended scope every time the closure is created:

In [39]:
def pow(n):
    # n is local to pow
    def inner(x):
        return x ** n
    return inner

In this example, `n`, in the function `inner` is a free variable, so we have a closure that constains `inner` and the free variable `n`

In [40]:
square = pow(2)

In [41]:
square(5)

25

In [42]:
cube = pow(3)

In [43]:
cube(5)

125

We can see that the cell used for the free variable in both cases is different:

In [44]:
square.__closure__

(<cell at 0x11338e6b0: int object at 0x104ad6fd0>,)

In [45]:
cube.__closure__

(<cell at 0x11338e020: int object at 0x104ad6ff0>,)

In fact, these functions (`square`, and `cube`) are not the same functions, even though they were "created" from the same `power` function:

In [46]:
id(square), id(cube)

(4619924032, 4619912352)

### Beware!

Remeber when we said the captured variable is a reference established when the closure is created, but the value is looked up only once the function is called?

This can create very `subtle bugs` in our program.

Consider the following example where we want to create some functions that can add 1, 2, 3, 4 and to whatever is passed to them.

We could do the following:

In [47]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [48]:
add_1 = adder(1)

In [49]:
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [50]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

#### <font color=#a3be8c> But suppose we want to get a littel fancier and do it as follows:

In [51]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [53]:
adders

NameError: name 'adders' is not defined

In [54]:
adders = create_adders()

> Technically we have 4 functions in the `adders` list:

In [55]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

The first one should add 1 to the value we pass it.  
The second should add 2, and so on

In [56]:
adders[3](10)

14

In [59]:
adders[3]

<function __main__.create_adders.<locals>.<lambda>(x)>

In [60]:
adders[3]()

TypeError: create_adders.<locals>.<lambda>() missing 1 required positional argument: 'x'

Yep!  
That works for the 4th function.

In [61]:
adders[0](10)

14

##### <font color=#bf616a> Oh, no! What Happened?

In fact we get the same behavior from every one of those functions:

In [63]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(14, 14, 14, 14)

##### <font color=#ebcb8b> Remember what we said about the variable is captured and when the value is looked up?

When the lambdas are created in their `n` is the `n` used in the loop - the same `n`!!</font>

In [64]:
adders[0].__code__.co_freevars

('n',)

In [70]:
adders[0].__closure__

(<cell at 0x11338e380: int object at 0x104ad7010>,)

In [67]:
adders[1].__closure__

(<cell at 0x11338e380: int object at 0x104ad7010>,)

In [71]:
adders[2].__closure__

(<cell at 0x11338e380: int object at 0x104ad7010>,)

In [72]:
adders[3].__closure__

(<cell at 0x11338e380: int object at 0x104ad7010>,)

##### <font color=#8fbcbb> The above closures points the same object.
##### <font color=#88c0d0> The above closures points the same object.
##### <font color=#81a1c1> The above closures points the same object.
##### <font color=#5e81ac> The above closures points the same object.

So, by the time we call `adders[i]`, the free variable `n` (shared between all adders) is set to 4.

In [73]:
hex(id(4))

'0x104ad7010'

<font color=#88c0d0> As we can see the memory address fo the singleton integer 4, is what cell is pointing to.

If we want to use a loop to do this and not end up using the same cell for each of the free variables, we can use a simple trick that forces the evaluation of `n` at the time the closure is created, instead of what the closure function is evaluated.

We can do this by createing a parameter for `n` in our lambda whose default value is the current value of `n` - remember from an earlier video that parametttter defaults are evaluated when the function is created, not called.

In [75]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders

Aaron的解釋：

``` python
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders
```

### 參數解析
``` python
lambda x, step=n: x + step
```

- x 是顯式傳入的參數（執行時需要提供）。
- step=n 是一個帶預設值的參數，如果 step 沒有被顯式提供，則它的值將是 n。

#### 為什麼 step=n 不是自由變數？
如果 lambda 內部只寫：

```python
lambda x: x + n
```
則 n 是一個「自由變數」，這表示 n 的值不是在 lambda 函數被創建時決定的，而是在 lambda 被執行時從外部作用域取得。
問題是，當 create_adders() 的 for 迴圈跑完後，`n 會變成最後一個值（4），這樣所有 lambda 函式都會使用 n=4。`

##### 而加上 step=n 之後：
``` python
lambda x, step=n: x + step
```
- step=n 這一部分的作用是讓 step 在 lambda 創建時 就綁定當前的 n 值，而不是等到 lambda 執行時才查找 n。
- 這樣，每個 lambda 都會擁有當前 n 值的獨立拷貝，而不會在 for 迴圈結束後統一變成 4。

### 示範

In [81]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders

In [82]:
adders = create_adders()

In [83]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x, step=1)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=2)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=3)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=4)>]

In [84]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(11, 12, 13, 14)

這證明 `step=n` 確保了每個 lambda 函式都正確記住了當時的 `n` 值。

---

### **簡單比喻**
假設 `n` 是一個快遞包裹，`step=n` 就像是在 lambda 創建時把包裹封存，確保裡面的內容不會變。  
如果沒有 `step=n`，就像 lambda 只是寫了一個「查詢單號」，等到快遞被查詢時，才發現所有查詢結果都是最後一個值 `4`。

這就是為什麼 `step=n` 能夠解決「變數捕獲問題」，並確保每個 lambda 取得不同的 `n` 值。

In [87]:
def create_adders():
    adders = []  
    for n in range(1, 5):  
        adders.append(lambda x, step=n: x + step)  # 確保 n 的值正確綁定
    return adders

# 創建加法器列表
adders = create_adders()
adders


[<function __main__.create_adders.<locals>.<lambda>(x, step=1)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=2)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=3)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=4)>]

In [88]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(11, 12, 13, 14)

In [90]:
# 測試每個 lambda 函式
print(adders[0](100))
print(adders[1](100))
print(adders[2](100))
print(adders[3](100))

101
102
103
104


In [91]:
adders[0].__closure__

Why aren't we getting anything in the closure?

What about free variables?

In [92]:
adders[0].__code__.co_freevars

()

Hmm, nothing either...Why?

Well, look at the lambda in that loop.

Does it reference the variable `n` (other than in the default value)

`No`! Hence, `n` is not a free variable in this case, and our lambda is just a plain lambda, `not a closuree`

And this code will now work as expectd:

In [93]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(11, 12, 13, 14)

We just need to understand that since the default values are evaluated when the function (lambda in this case) is created, the then-current `n` value is assigned to the local variable `step`.

So `step` will not change every time the lambda is called, and since n is not referenced inside the function (and therefore evaluated when the lambda is called). `n` is not a free variable.

### <font color=darkseagreen> Nested Closures

We can also next closures, as can be seen in this example:

In [94]:
def incrementer(n): 
    def inner(start):
        current = start
        def inc():
            a = 10 # local var
            nonlocal current
            current += n
            return current
        return inc
    return inner

In [95]:
fn = incrementer(3)

In [96]:
fn

<function __main__.incrementer.<locals>.inner(start)>

In [97]:
fn.__code__.co_freevars

('n',)

In [98]:
fn.__closure__

(<cell at 0x11522c940: int object at 0x104ad6ff0>,)

In [99]:
inc_2 = fn(100)

In [100]:
inc_2

<function __main__.incrementer.<locals>.inner.<locals>.inc()>

In [102]:
inc_2.__code__.co_freevars

('current', 'n')

In [103]:
inc_2.__closure__

(<cell at 0x11522c9d0: int object at 0x104ad7c10>,
 <cell at 0x11522c940: int object at 0x104ad6ff0>)

Here you can see that the second free variable `n`, is pointing to the same cell as free variable in `fn`.

Note that `a` is a local variable, and is not considered a free variable.

And we can call the closures as follows:

In [104]:
inc_2()

103

In [105]:
inc_2()

106

In [107]:
inc_3 = incrementer(3)(1000)

In [108]:
inc_3()

1003

In [109]:
inc_3()

1006

Let's examine that concept of a cell to create an indirect reference for variables that are in multiple scopes.

In [67]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [68]:
fn = outer()

In [69]:
fn.__code__.co_freevars

('x',)

As we can see, `x` is a free variable in the closure.

In [70]:
fn.__closure__

(<cell at 0x109ba8f40: str object at 0x10749bd30>,)

Here we see that the free variable x is actually a reference to a cell object that is itself a reference to a string object.

Let's see what the memory address of `x` is in the outer function and the inner function. To be sure string interning does not play a role, I am going to use an object that we know Python will not automatically intern, like a list.

In [71]:
def outer():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    def inner():
        print('inner:', hex(id(x)))
        print(x)
    return inner

In [72]:
fn = outer()

outer: 0x109be3180


In [73]:
fn.__closure__

(<cell at 0x109ba9fc0: list object at 0x109be3180>,)

In [74]:
fn()

inner: 0x109be3180
[1, 2, 3]


As you can see, each the memory address of `x` in `outer`, `inner` and the cell all point to the same object.

#### Modifying the Free Variable

We know we can modify nonlocal variables by using the `nonlocal` keyword. So the following will work:

In [75]:
def counter():
    count = 0 # local variable
    
    def inc():
        nonlocal count  # this is the count variable in counter
        count += 1
        return count
    return inc

In [76]:
c = counter()

In [77]:
c()

1

In [78]:
c()

2

##### Shared Extended Scopes

As we saw in the lecture, we can set up nonlocal variables in different inner functions that reference the same outer scope variable, i.e. we have a free variable that is shared between two closures. This works because both non local variables and the outer local variable all point back to the same cell object.

In [79]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

In [80]:
fn1, fn2 = outer()

In [81]:
fn1.__closure__, fn2.__closure__

((<cell at 0x109babd60: int object at 0x104b8ef90>,),
 (<cell at 0x109babd60: int object at 0x104b8ef90>,))

As you can see here, the `count` label points to the same cell.

In [82]:
fn1()

1

In [83]:
fn1()

2

In [84]:
fn2()

3

### Multiple Instances of Closures

Recall that **every** time a function is called, a **new** local scope is created.

In [85]:
from time import perf_counter

def func():
    x = perf_counter()
    print(x, id(x))

In [86]:
func()

72319.948119708 4455622736


In [87]:
func()

72319.951716125 4455623984


The same thing happens with closures, they have their own extended scope every time the closure is created:

In [88]:
def pow(n):
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    return inner

In this example, `n`, in the function `inner` is a free variable, so we have a closure that contains `inner` and the free variable `n`

In [89]:
square = pow(2)

In [90]:
square(5)

25

In [91]:
cube = pow(3)

In [92]:
cube(5)

125

We can see that the cell used for the free variable in both cases is **different**:

In [93]:
square.__closure__

(<cell at 0x109ba9930: int object at 0x104b8efd0>,)

In [94]:
cube.__closure__

(<cell at 0x109babd30: int object at 0x104b8eff0>,)

In fact, these functions (`square` and `cube`) are **not** the same functions, even though they were "created" from the same `power` function:

In [95]:
id(square), id(cube)

(4458496224, 4458496544)

### Beware!

Remember when I said the captured variable is a reference established when the closure is created, but the value is looked up only once the function is called?

This can create very subtle bugs in your program.

Consider the following example where we want to create some functions that can add 1, 2, 3, 4 and to whatever is passed to them.

We could do the following:

In [96]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [97]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [98]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

But suppose we want to get a little fancier and do it as follows:

In [99]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [100]:
adders = create_adders()

Now technically we have 4 functions in the `adders` list:

In [101]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

The first one should add 1 to the value we pass it, the second should add 2, and so on.

In [102]:
adders[3](10)

14

Yep, that works for the 4th function.

In [103]:
adders[0](10)

14

Uh Oh - what happened? In fact we get the same behavior from every one of those functions:

In [104]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(14, 14, 14, 14)

Remember what I said about when the variable is captured and when the value is looked up?

When the lambdas are **created** their `n` is the `n` used in the loop - the **same** `n`!!

In [105]:
adders[0].__code__.co_freevars

('n',)

In [106]:
adders[0].__closure__

(<cell at 0x109baa3e0: int object at 0x104b8f010>,)

In [107]:
adders[1].__closure__

(<cell at 0x109baa3e0: int object at 0x104b8f010>,)

In [108]:
adders[2].__closure__

(<cell at 0x109baa3e0: int object at 0x104b8f010>,)

In [109]:
adders[3].__closure__

(<cell at 0x109baa3e0: int object at 0x104b8f010>,)

So, by the time we call `adder[i]`, the free variable `n` (shared between all adders) is set to 4.

In [110]:
hex(id(4))

'0x104b8f010'

As we can see the memory address of the singleton integer 4, is what that cell is pointint to.

If you want to use a loop to do this and not end up using the same cell for each of the free variables, we can use a simple trick that forces the evaluation of `n` at the time the closure is **created**, instead of when the closure function is evaluated.

We can do this by creating a parameter for `n` in our lambda whose default value is the current value of `n` - remember from an earlier video that parameter defaults are avaluated when the function is created, not called.

In [111]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders

In [112]:
adders = create_adders()

In [113]:
adders[0].__closure__

Why aren't we getting anything in the closure? What about free variables?

In [114]:
adders[0].__code__.co_freevars

()

Hmm, nothing either... Why?

Well, look at the lambda in that loop. Does it reference the variable `n` (other than in the default value)? No. Hence, `n` is **not** a free variable in this case, and our lambda is just a plain lambda, not a closure.

And this code will now work as expected:

In [115]:
adders[0](10)

11

In [116]:
adders[1](10)

12

In [117]:
adders[2](10)

13

In [118]:
adders[3](10)

14

You just need to understand that since the default values are evaluated when the function (lambda in this case) is **created**, the then-current `n` value is assigned to the local variable `step`. So `step` will not change every time the lambda is called, and since n is not referenced inside the function (and therefore evaluated when the lambda is called), `n` is not a free variable.

#### Nested Closures

We can also nest closures, as can be seen in this example:

In [119]:
def incrementer(n):
    def inner(start):
        current = start
        def inc():
            a = 10  # local var
            nonlocal current
            current += n
            return current
        return inc
    return inner
        

In [120]:
fn = incrementer(2)

In [121]:
fn

<function __main__.incrementer.<locals>.inner(start)>

In [122]:
fn.__code__.co_freevars

('n',)

In [123]:
fn.__closure__

(<cell at 0x109ba84f0: int object at 0x104b8efd0>,)

In [124]:
inc_2 = fn(100)

In [125]:
inc_2

<function __main__.incrementer.<locals>.inner.<locals>.inc()>

In [126]:
inc_2.__code__.co_freevars

('current', 'n')

In [127]:
inc_2.__closure__

(<cell at 0x109ba8e20: int object at 0x104b8fc10>,
 <cell at 0x109ba84f0: int object at 0x104b8efd0>)

Here you can see that the second free variable `n`, is pointing to the same cell as the free variable in `fn`.

Note that **a** is a local variable, and is not considered a free variable.

And we can call the closures as follows:

In [128]:
inc_2()

102

In [129]:
inc_2()

104

In [130]:
inc_3 = incrementer(3)(200)

In [131]:
inc_3()

203

In [132]:
inc_3()

206